In [3]:
!pip install keras-tuner

     |████████████████████████████████| 71kB 5.6MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp36-none-any.whl size=78939 sha256=bdb48ab0ecca0e6753b3d1dd8f385eeedeee290fa649a91f094918d0428a12ce
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15358 sha256=7d55c831c8d26fd5fe6cf0bcd58a45b796280db5abef2edeca16df4f288144fb
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [4]:
import tensorflow as tf
import keras
from keras.layers import Conv2D, Dense, Flatten, MaxPooling2D
from keras.models import Sequential
import numpy as np 
import cv2
import os
import matplotlib.pyplot as plt
from kerastuner.tuners import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [5]:
#upload zip file onto google drive and unzip the file here to read local data
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
!unzip -uq "/content/drive/MyDrive/CatsDog_GoogleColab" -d "/content/drive/MyDrive/Colab Notebooks/CatsDog_GoogleColab"

In [6]:
path = "/content/drive/MyDrive/Colab Notebooks/CatsDog_GoogleColab/dataset/training_set"
categories = ['cats', 'dogs']
IMG_SIZE = 100

In [9]:
training = []

for category in categories:
    folder = os.path.join(path,category)
    label = categories.index(category)
    for img in os.listdir(folder):
        img_path = os.path.join(folder,img)
        img_array = cv2.imread(img_path)
        img_array = cv2.resize(img_array,(IMG_SIZE,IMG_SIZE))
        training.append([img_array,label])


In [11]:
import random
random.shuffle(training)

In [12]:
xtrain = [feature for feature, label in training] 
ytrain = [label for feature,label in training] 

In [13]:
xtrain = np.array(xtrain)
ytrain = np.array(ytrain)
xtrain = xtrain/255 

In [10]:
test = [] 
path = "/content/drive/MyDrive/Colab Notebooks/CatsDog_GoogleColab/dataset/test_set"


for category in categories:
    folder = os.path.join(path,category)
    label = categories.index(category)
    for img in os.listdir(folder):
        img_path = os.path.join(folder,img)
        img_array = cv2.imread(img_path)
        img_array = cv2.resize(img_array,(IMG_SIZE,IMG_SIZE))
        test.append([img_array,label])

In [15]:
xtest = [feature for feature, label in test] 
ytest = [label for feature,label in test] 

In [29]:
xtest = np.array(xtest)
ytest = np.array(ytest)

In [16]:
def build_model(hp):
    model = Sequential()
    model.add(Conv2D(hp.Int(f'input-unit',32,256,32), 3, input_shape=xtrain.shape[1:], activation='relu'))
    model.add(MaxPooling2D(2,2))

    for n in range(hp.Int('layer', 1, 4)):
        model.add(Conv2D(32, 3, activation='relu'))
    
    model.add(Flatten())
    model.add(Dense(2, activation='sigmoid'))

    model.compile(optimizer='adam',
                    loss = 'sparse_categorical_crossentropy',
                    metrics = ['accuracy'])

    return model

In [23]:
tuner = RandomSearch(
    build_model,
    objective='accuracy',
    max_trials = 10,
    executions_per_trial = 1,
    directory='my_dir',
    project_name='cat&dog'
)

tuner.search(xtrain, ytrain, epochs = 5, batch_size=64, validation_split=0.2)

Trial 10 Complete [00h 00m 34s]
accuracy: 0.8547374606132507

Best accuracy So Far: 0.9063926935195923
Total elapsed time: 00h 11m 20s
INFO:tensorflow:Oracle triggered exit


In [24]:
tuner.results_summary()

Results summary
Results in my_dir/cat&dog
Showing 10 best trials
Objective(name='accuracy', direction='max')
Trial summary
Hyperparameters:
input-unit: 64
layer: 2
Score: 0.9063926935195923
Trial summary
Hyperparameters:
input-unit: 96
layer: 2
Score: 0.8905060887336731
Trial summary
Hyperparameters:
input-unit: 160
layer: 2
Score: 0.8894596695899963
Trial summary
Hyperparameters:
input-unit: 96
layer: 1
Score: 0.8587328791618347
Trial summary
Hyperparameters:
input-unit: 32
layer: 3
Score: 0.8547374606132507
Trial summary
Hyperparameters:
input-unit: 128
layer: 3
Score: 0.8291476368904114
Trial summary
Hyperparameters:
input-unit: 96
layer: 3
Score: 0.823630154132843
Trial summary
Hyperparameters:
input-unit: 224
layer: 3
Score: 0.8074581623077393
Trial summary
Hyperparameters:
input-unit: 224
layer: 2
Score: 0.7512366771697998
Trial summary
Hyperparameters:
input-unit: 192
layer: 4
Score: 0.6256659030914307


In [30]:
best_model = tuner.get_best_models(num_models=1)[0]
best_model.evaluate(xtest, ytest)


63/63 [==============================] - 1s 10ms/step - loss: 175.7882 - accuracy: 0.6841


[159.7865753173828, 0.6955000162124634]